In [1]:
!pip install notebook pandas geopandas shapely duckdb jupysql duckdb-engine

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.6/323.6 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 102.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.1/95.1 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.8/192.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.8/27.8 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 94.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.1/463.1 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 583.9/583.9 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.7/96.7

In [2]:
# import our toolkit
import pandas as pd
import geopandas as gpd
from shapely import wkt
import duckdb

In [3]:
# no need to import duckdb_engine, JupySQL will auto-detect driver
# load (or reload) jupysql Jupyter extension to create SQL cells
%reload_ext sql

In [4]:
# DuckDB in-memory database
%sql duckdb://

Connecting to 'duckdb://'

In [5]:
# Run if error
#%sql ROLLBACK

In [6]:
%sql INSTALL spatial;
%sql LOAD spatial;

Running query in 'duckdb://'

Running query in 'duckdb://'

Success


In [7]:
%%sql
COPY (
    SELECT
        id,
        level,
        height,
        names.primary AS primary_name,
        sources[1].dataset AS primary_source,
        geometry
    FROM read_parquet(
        's3://overturemaps-us-west-2/release/2025-05-21.0/theme=buildings/type=*/*',
        hive_partitioning=1
    )
    WHERE
        bbox.xmin > -121.9135
        AND bbox.xmax < -121.8915  -- extended farther east
        AND bbox.ymin > 37.4080
        AND bbox.ymax < 37.4236   -- extended farther north
) TO 'greatmall_buildings.geojson' WITH (FORMAT GDAL, DRIVER 'GeoJSON');

Running query in 'duckdb://'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Count


In [8]:
%%sql
COPY(
    SELECT
        id,
        names.primary AS name,
        CAST(addresses AS JSON) AS addresses,
        confidence,
        geometry
    FROM read_parquet(
        's3://overturemaps-us-west-2/release/2025-04-23.0/theme=places/type=place/*',
        filename=true,
        hive_partitioning=1
    )
    WHERE bbox.xmin BETWEEN -121.91 AND -121.87
      AND bbox.ymin BETWEEN 37.41 AND 37.42
) TO 'greatmall_pois.geojson' WITH (FORMAT GDAL, DRIVER 'GeoJSON');

Running query in 'duckdb://'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Count


In [9]:
# Load and project to meters
pois = gpd.read_file("greatmall_pois.geojson").to_crs(epsg=32610)
buildings = gpd.read_file("greatmall_buildings.geojson").to_crs(epsg=32610)

In [11]:
buffered = pois.copy()
buffered["geometry"] = buffered.geometry.buffer(50)
candidates = gpd.sjoin(buildings, buffered, how="inner", predicate="intersects")

In [12]:
def find_best_building(poi_row):
    candidates_for_point = candidates[candidates.index_right == poi_row.name]
    if candidates_for_point.empty:
        return poi_row.geometry  # no match, leave unchanged
    # pick the closest one
    candidates_for_point["dist"] = candidates_for_point.geometry.centroid.distance(poi_row.geometry)
    best = candidates_for_point.sort_values("dist").iloc[0]
    return best.geometry.centroid  # move to the building centroid

In [15]:
pois["new_geometry"] = pois.apply(find_best_building, axis=1)
pois = pois.set_geometry("new_geometry")               # use new geometry
pois = pois.set_crs(epsg=32610, inplace=False)         # assign UTM CRS
pois = pois.to_crs(epsg=4326)                          # convert back to WGS84
pois = pois.drop(columns=["geometry"])                 # drop old column
pois = pois.rename(columns={"new_geometry": "geometry"})
pois = pois.set_geometry("geometry")                   # activate renamed geometry

In [16]:
pois.to_file("address_matched_snapped_pois.geojson", driver="GeoJSON")